In [2]:
import copy

import numpy as np

import scipy.integrate as integrate
from scipy import constants
from scipy import interpolate

# Plotting
import matplotlib.pyplot as plt
from matplotlib import scale as mscale
from matplotlib.transforms import Transform
from matplotlib.ticker import FixedLocator

import seaborn as sns

# CLASS

from classy import Class

In [3]:
_GN_ = constants.gravitational_constant # Meter^3 Kilogram^-1 Second^-2 
_c_light_ = constants.c # Meter Second^-1
_eV_ = constants.eV; # Meter^2 Kilogram Second^-2
_kB_ = constants.Boltzmann # Joule Kelvin^-1
_hP_ = constants.Planck # Joule Second 
_aB_ = 8*np.pi**5*_kB_**4/(15*_hP_**3*_c_light_**3) #Joule Kelvin^-4 Meter-3

#Conversions
_Meter_per_MPc_ = constants.mega*constants.parsec
H_0_std=100 * constants.kilo/_Meter_per_MPc_

#Class Default Values
Tcmb_pk = 2.7255 # Kelvin 
Nur_pk = 3.046;
h_pk = 0.67556;
omega_b_pk = 0.022032;
omega_cdm_pk = 0.12038;

Omega_b_pk = omega_b_pk/(h_pk**2)
Omega_cdm_pk = omega_cdm_pk/(h_pk**2)

H_0_pk= h_pk* H_0_std
rho_crit_pk= 3* H_0_pk**2 /(8*np.pi*_GN_)


def a_of_z(z,a_0=1.):
    return a_0/(1.+z)

In [4]:
class cosmic_fluid(object):
    """A Simple fluid model for cosmic material.
        
        Naught values corrispond to today.
        """
    def p(self,a):
        return self.rho(a) * self.w(a)
 
        
class cosmic_fluid_const_w(cosmic_fluid):
    """A cosmic fluid with constant w(a)"""
    
    def __init__(self,rho_ref,w,a_0=1.,a_ref=1.):
        self.w = lambda a=a_0: w
        self.a_0 = a_0
        self.a_ref = a_ref
        self.rho_0 = rho_ref * (a_ref/a_0)**(3*(1+w))
        
        
    def rho(self,a):
        return self.rho_0*(self.a_0/a)**(3*(1+ self.w()))
    

    
class cosmic_fluid_spline_w(cosmic_fluid):
    """A cosmic fluid with a given spline made with numpy.interpolate.splrep for w(a)"""
    
    def __init__(self,rho_ref,spline_log10a_vals,spline_w_vals,a_0=1.,a_ref=1.):
        
        #Store spline knots and create spline representation
        self.spline_log10a_vals=spline_log10a_vals
        self.spline_w_vals=spline_w_vals
        self.w_spline_rep=interpolate.splrep(spline_log10a_vals,spline_w_vals, s=0)
        
        self.w = lambda a:interpolate.splev(np.log10(a), self.w_spline_rep, der=0)
        self.a_0 = a_0
        self.a_ref= a_ref
        self.rho_0 = rho_ref * (a_ref/a_0)**3* 10**(-3*interpolate.splint(np.log10(a_ref),np.log10(a_0),self.w_spline_rep))
        
    def rho(self,a):
        return self.rho_0*(self.a_0/a)**3 * 10**(-3*interpolate.splint(np.log10(self.a_0),np.log10(a),self.w_spline_rep))
    

In [5]:
def rho_0_gamma(T=Tcmb_pk):
    """The derived rho_0 for photos from photon temperatures"""
    return _aB_ / _c_light_**2 * T**4

In [18]:
def test_spline_vs_constant():
    """A test of the spline class vs constant w class"""
    
    #Spline knots domain values
    log10a_vals= np.arange(-14.5,0.5,0.1)
    #Spline knots range values
    w_vals=np.array([1/3]*len(log10a_vals))
    
    #Constant w model
    gamma= cosmic_fluid_const_w(rho_0_gamma(),1/3)
    gamma_spl= cosmic_fluid_spline_w(rho_0_gamma(),log10a_vals,w_vals)

    #Ensure rho,p,w have evolved correctly
    tol=10**-15
    a_test=10**np.random.randint(-14,0)
    
    assert abs((gamma.rho(a_test)-gamma_spl.rho(a_test))/gamma.rho(a_test))<tol, f"rho at {np.log10(a_test)} differs beyond tolerance, via con:{gamma.rho(a_test)} and via spl:{gamma_spl.rho(a_test)} "
    assert abs((gamma.p(a_test)-gamma_spl.p(a_test))/gamma.p(a_test))<tol,     f"p at {np.log10(a_test)} differs beyond tolerance"
    assert abs((gamma.w(a_test)-gamma_spl.w(a_test))/gamma.w(a_test))<tol,     f"w at {np.log10(a_test)} differs beyond tolerance"
    "Pass"

test_spline_vs_constant()